# Amazon SageMaker Studio + SSH + IDE Integration

Run this notebook every time when you start a new SageMaker Studio App.

## Environment setup

First, install the SageMaker SSH Helper library into the kernel app. An old version of AWS CLI v1 have to be uninstalled, if present.

In [ ]:
%%sh
pip3 uninstall -y -q awscli
pip3 install -q -U sagemaker-ssh-helper
pip3 freeze | grep sagemaker-ssh-helper

Now run the next cell to get the SageMaker Studio kernel gateway app name, user profile name and domain ID which you will need later to connect to the from your local machine with `sm-ssh connect` command.

In [ ]:
%%sh
sm-ssh-ide get-metadata

The next command will install the software for remote access, including SSH daemon and VNC server and XFCE4 desktop. If you don't want to run a VNC server or Jupyter notebook, pass the flag `--ssh-only` (comment the first line and uncomment the second).

In [ ]:
%%sh
sm-ssh-ide configure
#sm-ssh-ide configure --ssh-only

Make sure the above command ends with the successful message:

`sm-ssh-ide: Configuration complete.`

If you don't see this message, your configuration is not complete and the next steps will not work. One of the most common problems is that installation runs out of memory and silently terminates. In this case, switch to an instance with at least 8 GB of RAM, e.g. `ml.m5.large` .

## (Optional) Remote desktop / VNC setup

If you don't plan to use VNC and run any GUI apps inside SageMaker Studio, skip to the next section or run the cells as is (safe defaults).

Choose your VNC password. It's safe to leave it as is, because your connection will be already secured with SSH and SSM. You need some password, because some VNC clients do not allow passwordless connections.

If you already configured password from another kernel app, this cell will take no effect (the encrypted password file will be shared between apps).

In [ ]:
%%sh
VNC_PASSWORD="123456"  # replace with your password

sm-ssh-ide set-vnc-password "$VNC_PASSWORD"

Specify JetBrains License Server host name in `JB_LICENSE_SERVER_HOST` variable. Leave it as is if you don't use one, e.g. use PyCharm Community or VS Code, or already configured one from another kernel app (the value will be shared between apps).


In [ ]:
%%sh
JB_LICENSE_SERVER_HOST="jetbrains-license-server.example.com"  # replace with your JetBrains License Server host name

sm-ssh-ide set-jb-license-server "$JB_LICENSE_SERVER_HOST"

## SSM setup

In the next cell you need to put your local user id as the parameter to `init-ssm`. It will help to tag the managed instance with your [aws:userid](https://docs.aws.amazon.com/IAM/latest/UserGuide/reference_policies_variables.html#principaltable) as the principal owner. You can find the id by running locally the `aws sts get-caller-identity` command as the value of the returned `UserId`.

If you already configured the id from another kernel app, leave the value as is (it will be loaded from the shared configuration file automatically).

In [ ]:
%%sh
LOCAL_USER_ID="AIDACKCEVSQ6C2EXAMPLE:terry@SSO"  # replace with your local UserId

sm-ssh-ide set-local-user-id "$LOCAL_USER_ID"

The next command will initialize the Systems Manager Agent and register the instance.

In [ ]:
%%sh
sm-ssh-ide init-ssm

## Run SSM Agent & Services

The `start` command will run the background applications for remote access – SSH daemon, VNC server and Jupyter notebook. The `stop` command helps with cleanup from previous runs (see the next section).

If you passed the flag `--ssh-only` to the `configure` command earlier, a VNC server and Jupyter notebook will not be started.

In [ ]:
%%sh
sm-ssh-ide stop
sm-ssh-ide start

Now execute `ssm-agent` command and keep it running (do not stop the kernel). To validate that the SageMaker Studio app has connected to SSM, go to the Systems Manager Console > Fleet Manager and check that a new managed node appeared in the list and got the "SSM Agent ping status" as Online.

In [ ]:
%%sh
sm-ssh-ide ssm-agent

The above cell will run SSM Agent Daemon infinitely until it stopped.

At this point, complete the instructions for your local machine described in [README.md](README.md#studio).

To stop the Agent, interrupt the kernel.

## Stop SSM Agent & Services

Execute `stop` command to gracefully shut down the SSM Agent Daemon, SSH tunnel, port forwarding and other services.
It's useful when you want to restart the services. If you don't call this command before calling `start` again, you will get a message *Already running on this instance? Call stop first.*

In [ ]:
%%sh
sm-ssh-ide stop